In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stat
import scipy.io as scipio
import matplotlib.pyplot as plt
%matplotlib inline
import plotly as ly
import datetime
import mapbox
import folium

In [139]:
## Load date and holiday data
date_info = pd.DataFrame(pd.read_csv('data/date_info.csv'))
## Load air data
air_reserve = pd.DataFrame(pd.read_csv('data/air_reserve.csv'))
air_store = pd.DataFrame(pd.read_csv('data/air_store_info.csv'))
air_visit = pd.DataFrame(pd.read_csv('data/air_visit_data.csv'))
## AirREGI Restaurants with Nearest Active Station - add second and third
air_nearest_weather = pd.DataFrame(pd.read_csv('data/rrv-weather-data/air_store_info_with_nearest_active_station.csv'))
## add second and third closest distances
station_distances = pd.DataFrame(pd.read_csv('data/rrv-weather-data/air_station_distances.csv'))

In [27]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


## Restaurant Data 

3 closest weather stations, genre, latlong, and area name

In [30]:
air_nearest_weather.head()
air_byrestindex = air_nearest_weather.set_index(air_nearest_weather["air_store_id"],drop=True)
air_byrestindex.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle
air_store_id,,,,,,,,,,,,
air_0f0cdeee6c9bf3d7,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
air_7cc17a324ae5c7dc,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
air_fee8dcf4d619598e,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
air_a17f0778617c76e2,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
air_83db5aff8f50478e,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,"""35.6580681""","""139.7515992""",tokyo__tokyo-kana__tonokyo,35.691667,139.750000,3.730672,3.739835


In [15]:
len(air_store)

829

In [16]:
len(air_nearest_weather)

829

In [36]:
station_latlong = station_distances[["station_id", "station_latitude", "station_longitude"]]
# station_latlong = station_latlong.set_index(station_latlong["station_id"])
# station_latlong.head()

In [37]:
station_latlong.head()

,station_id,station_latitude,station_longitude
0,aichi__ai-xi-kana__isaai,35.216667,136.698333
1,aichi__aki-kana__azo,35.078333,137.413333
2,aichi__centrair-kana__centrair,34.858333,136.805000
3,aichi__chapel-mountain-kana__chaw-yama,35.220000,137.660000
4,aichi__gamagori-kana__gamagori,34.845000,137.216667


In [49]:
from vincenty import vincenty

station_lats = np.array(station_latlong["station_latitude"])
station_longs = np.array(station_latlong["station_longitude"])
station_latlong["latlong_m"] = list(np.vstack((station_lats, station_longs)).T)

/Users/derin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [50]:
station_latlong.head()

,station_id,station_latitude,station_longitude,latlong_m
0,aichi__ai-xi-kana__isaai,35.216667,136.698333,"[35.2166666667, 136.698333333]"
1,aichi__aki-kana__azo,35.078333,137.413333,"[35.0783333333, 137.413333333]"
2,aichi__centrair-kana__centrair,34.858333,136.805000,"[34.8583333333, 136.805]"
3,aichi__chapel-mountain-kana__chaw-yama,35.220000,137.660000,"[35.22, 137.66]"
4,aichi__gamagori-kana__gamagori,34.845000,137.216667,"[34.845, 137.216666667]"


In [54]:
rest_lats = np.array(air_nearest_weather["latitude"])
rest_longs = np.array(air_nearest_weather["longitude"])
air_nearest_weather["latlong_m"] = list(np.vstack((rest_lats, rest_longs)).T)

In [56]:
air_nearest_weather.head(3)

,air_store_id,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle,latlong_m
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882,"[34.6951242, 135.1978525]"
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882,"[34.6951242, 135.1978525]"
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882,"[34.6951242, 135.1978525]"


In [74]:
air_nearest_weather[air_nearest_weather["air_store_id"]=="air_fee8dcf4d619598e"]["latlong_m"].index[0]

2

In [75]:
station_latlong["air_fee8dcf4d619598e"] = station_latlong["latlong_m"].map(lambda x: vincenty(x, air_nearest_weather[air_nearest_weather["air_store_id"]=="air_fee8dcf4d619598e"]["latlong_m"][air_nearest_weather[air_nearest_weather["air_store_id"]=="air_fee8dcf4d619598e"]["latlong_m"].index[0]]))



/Users/derin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [188]:
station_latlong.head()

,station_id,station_latitude,station_longitude,latlong_m,air_0f0cdeee6c9bf3d7,air_fee8dcf4d619598e,air_7cc17a324ae5c7dc,air_a17f0778617c76e2,air_83db5aff8f50478e,air_99c3eae84130c1cb,...,air_e9ebf7fc520ac76a,air_f4936b91c9addbf0,air_47beaffd3806c979,air_edd5e3d696a5811b,air_6b65745d432fd77f,air_9bf595ef095572fb,air_764f71040a413d4d,air_10bbe8acd943d8f6,air_7514d90009613cd6,air_c8fe396d6c46275d
0,aichi__ai-xi-kana__isaai,35.216667,136.698333,"[35.2166666667, 136.698333333]",148.760474,148.760474,148.760474,148.760474,281.515494,281.515494,...,279.581130,279.581130,282.353537,282.353537,277.332264,277.332264,277.332264,277.332264,957.952227,957.952227
1,aichi__aki-kana__azo,35.078333,137.413333,"[35.0783333333, 137.413333333]",206.934423,206.934423,206.934423,206.934423,222.009692,222.009692,...,221.478196,221.478196,223.324943,223.324943,217.995118,217.995118,217.995118,217.995118,948.288407,948.288407
2,aichi__centrair-kana__centrair,34.858333,136.805000,"[34.8583333333, 136.805]",148.219287,148.219287,148.219287,148.219287,282.425838,282.425838,...,282.084449,282.084449,283.818835,283.818835,278.447541,278.447541,278.447541,278.447541,990.897462,990.897462
3,aichi__chapel-mountain-kana__chaw-yama,35.220000,137.660000,"[35.22, 137.66]",232.290097,232.290097,232.290097,232.290097,196.028930,196.028930,...,195.069818,195.069818,197.189067,197.189067,191.950202,191.950202,191.950202,191.950202,926.006772,926.006772
4,aichi__gamagori-kana__gamagori,34.845000,137.216667,"[34.845, 137.216666667]",185.551102,185.551102,185.551102,185.551102,247.697421,247.697421,...,247.994878,247.994878,249.297655,249.297655,243.820606,243.820606,243.820606,243.820606,978.751268,978.751268


In [88]:
# station_latlong["air_fee8dcf4d619598e"] = station_latlong["latlong_m"].map(lambda x: vincenty(x, air_nearest_weather[air_nearest_weather["air_store_id"]=="air_fee8dcf4d619598e"]["latlong_m"][0]))

def get_distances(station_data, rest_data):
    for rest in list(rest_data["air_store_id"])[-300:-1]:
        station_data[rest] = station_data["latlong_m"].\
        map(lambda x: vincenty(x, rest_data[rest_data["air_store_id"]==rest]["latlong_m"][rest_data[rest_data["air_store_id"]==rest]["latlong_m"].index[0]]))
    return station_data

# get_distances(station_latlong, air_nearest_weather)


In [190]:
station_latlong["air_c8fe396d6c46275d"] = station_latlong["latlong_m"].\
        map(lambda x: vincenty(x, air_nearest_weather[air_nearest_weather["air_store_id"]=="air_c8fe396d6c46275d"]["latlong_m"][air_nearest_weather[air_nearest_weather["air_store_id"]=="air_c8fe396d6c46275d"]["latlong_m"].index[0]]))
# station_latlong["air_0f0cdeee6c9bf3d7"] = station_latlong["latlong_m"].\
#         map(lambda x: vincenty(x, air_nearest_weather[air_nearest_weather["air_store_id"]=="air_0f0cdeee6c9bf3d7"]["latlong_m"][air_nearest_weather[air_nearest_weather["air_store_id"]=="air_0f0cdeee6c9bf3d7"]["latlong_m"].index[0]]))
    
    

KeyError: 'latlong_m'

In [104]:
station_latlong.to_csv("station_rest_distances.csv")

In [105]:
## filter this
station_rest_latlong = pd.DataFrame(pd.read_csv("station_rest_distances.csv"))
station_rest_latlong.head()

,Unnamed: 0,station_id,station_latitude,station_longitude,latlong_m,air_0f0cdeee6c9bf3d7,air_fee8dcf4d619598e,air_7cc17a324ae5c7dc,air_a17f0778617c76e2,air_83db5aff8f50478e,...,air_e9ebf7fc520ac76a,air_f4936b91c9addbf0,air_47beaffd3806c979,air_edd5e3d696a5811b,air_6b65745d432fd77f,air_9bf595ef095572fb,air_764f71040a413d4d,air_10bbe8acd943d8f6,air_7514d90009613cd6,air_c8fe396d6c46275d
0,0,aichi__ai-xi-kana__isaai,35.216667,136.698333,[ 35.21666667 136.69833333],148.760474,148.760474,148.760474,148.760474,281.515494,...,279.581130,279.581130,282.353537,282.353537,277.332264,277.332264,277.332264,277.332264,957.952227,957.952227
1,1,aichi__aki-kana__azo,35.078333,137.413333,[ 35.07833333 137.41333333],206.934423,206.934423,206.934423,206.934423,222.009692,...,221.478196,221.478196,223.324943,223.324943,217.995118,217.995118,217.995118,217.995118,948.288407,948.288407
2,2,aichi__centrair-kana__centrair,34.858333,136.805000,[ 34.85833333 136.805 ],148.219287,148.219287,148.219287,148.219287,282.425838,...,282.084449,282.084449,283.818835,283.818835,278.447541,278.447541,278.447541,278.447541,990.897462,990.897462
3,3,aichi__chapel-mountain-kana__chaw-yama,35.220000,137.660000,[ 35.22 137.66],232.290097,232.290097,232.290097,232.290097,196.028930,...,195.069818,195.069818,197.189067,197.189067,191.950202,191.950202,191.950202,191.950202,926.006772,926.006772
4,4,aichi__gamagori-kana__gamagori,34.845000,137.216667,[ 34.845 137.21666667],185.551102,185.551102,185.551102,185.551102,247.697421,...,247.994878,247.994878,249.297655,249.297655,243.820606,243.820606,243.820606,243.820606,978.751268,978.751268


In [89]:
air_nearest_weather.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle,latlong_m
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882,"[34.6951242, 135.1978525]"
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882,"[34.6951242, 135.1978525]"
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882,"[34.6951242, 135.1978525]"
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882,"[34.6951242, 135.1978525]"
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,"""35.6580681""","""139.7515992""",tokyo__tokyo-kana__tonokyo,35.691667,139.750000,3.730672,3.739835,"[35.6580681, 139.7515992]"


In [96]:
air_nearest_weather = air_nearest_weather.drop(air_nearest_weather["air_store_id"]=="air_c8fe396d6c46275d",0)

In [99]:
air_nearest_weather[air_nearest_weather["air_store_id"]=="air_c8fe396d6c46275d"]

,air_store_id,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle,latlong_m
828,air_c8fe396d6c46275d,Karaoke/Party,Hokkaidō Sapporo-shi Minami 3 Jōnishi,43.05546,141.340956,"""43.0554601""","""141.3409556""",hokkaido_ishikari__sapporo-katakana__satporo,43.06,141.328333,1.145298,1.143351,"[43.0554601, 141.3409556]"


In [110]:
station_rest_latlong.head()

,Unnamed: 0,station_id,station_latitude,station_longitude,latlong_m,air_0f0cdeee6c9bf3d7,air_fee8dcf4d619598e,air_7cc17a324ae5c7dc,air_a17f0778617c76e2,air_83db5aff8f50478e,...,air_e9ebf7fc520ac76a,air_f4936b91c9addbf0,air_47beaffd3806c979,air_edd5e3d696a5811b,air_6b65745d432fd77f,air_9bf595ef095572fb,air_764f71040a413d4d,air_10bbe8acd943d8f6,air_7514d90009613cd6,air_c8fe396d6c46275d
0,0,aichi__ai-xi-kana__isaai,35.216667,136.698333,[ 35.21666667 136.69833333],148.760474,148.760474,148.760474,148.760474,281.515494,...,279.581130,279.581130,282.353537,282.353537,277.332264,277.332264,277.332264,277.332264,957.952227,957.952227
1,1,aichi__aki-kana__azo,35.078333,137.413333,[ 35.07833333 137.41333333],206.934423,206.934423,206.934423,206.934423,222.009692,...,221.478196,221.478196,223.324943,223.324943,217.995118,217.995118,217.995118,217.995118,948.288407,948.288407
2,2,aichi__centrair-kana__centrair,34.858333,136.805000,[ 34.85833333 136.805 ],148.219287,148.219287,148.219287,148.219287,282.425838,...,282.084449,282.084449,283.818835,283.818835,278.447541,278.447541,278.447541,278.447541,990.897462,990.897462
3,3,aichi__chapel-mountain-kana__chaw-yama,35.220000,137.660000,[ 35.22 137.66],232.290097,232.290097,232.290097,232.290097,196.028930,...,195.069818,195.069818,197.189067,197.189067,191.950202,191.950202,191.950202,191.950202,926.006772,926.006772
4,4,aichi__gamagori-kana__gamagori,34.845000,137.216667,[ 34.845 137.21666667],185.551102,185.551102,185.551102,185.551102,247.697421,...,247.994878,247.994878,249.297655,249.297655,243.820606,243.820606,243.820606,243.820606,978.751268,978.751268


In [113]:
air_nearest_weather["1_close_st"] = air_nearest_weather["air_store_id"].map(lambda x: list(station_rest_latlong[["station_id",str(x)]].sort_values(by=x)["station_id"])[0])
air_nearest_weather["2_close_st"] = air_nearest_weather["air_store_id"].map(lambda x: list(station_rest_latlong[["station_id",str(x)]].sort_values(by=x)["station_id"])[1])
air_nearest_weather["3_close_st"] = air_nearest_weather["air_store_id"].map(lambda x: list(station_rest_latlong[["station_id",str(x)]].sort_values(by=x)["station_id"])[2])
air_nearest_weather["4_close_st"] = air_nearest_weather["air_store_id"].map(lambda x: list(station_rest_latlong[["station_id",str(x)]].sort_values(by=x)["station_id"])[3])
air_nearest_weather["5_close_st"] = air_nearest_weather["air_store_id"].map(lambda x: list(station_rest_latlong[["station_id",str(x)]].sort_values(by=x)["station_id"])[4])

In [288]:
pd.DataFrame(pd.read_csv("station_rest_distances.csv")).sort_values(by="air_0f0cdeee6c9bf3d7")

,Unnamed: 0,station_id,station_latitude,station_longitude,latlong_m,air_0f0cdeee6c9bf3d7,air_fee8dcf4d619598e,air_7cc17a324ae5c7dc,air_a17f0778617c76e2,air_83db5aff8f50478e,...,air_e9ebf7fc520ac76a,air_f4936b91c9addbf0,air_47beaffd3806c979,air_edd5e3d696a5811b,air_6b65745d432fd77f,air_9bf595ef095572fb,air_764f71040a413d4d,air_10bbe8acd943d8f6,air_7514d90009613cd6,air_c8fe396d6c46275d
655,655,hyogo__kobe-kana__koube,34.696667,135.211667,[ 34.69666667 135.21166667],1.277232,1.277232,1.277232,1.277232,427.032954,...,425.792170,425.792170,428.132922,428.132922,422.936501,422.936501,422.936501,422.936501,1068.822215,1068.822215
654,654,hyogo__kobe-airport-kana__kobuteuko,34.631667,135.223333,[ 34.63166667 135.22333333],7.416958,7.416958,7.416958,7.416958,428.023269,...,426.967634,426.967634,429.187810,429.187810,423.950972,423.950972,423.950972,423.950972,1074.700077,1074.700077
666,666,hyogo__rokko-mountain-kana__rokkou-san____2006...,34.763333,135.270000,[ 34.76333333 135.27 ],10.045909,10.045909,10.045909,10.045909,419.921994,...,418.535608,418.535608,420.970397,420.970397,415.807119,415.807119,415.807119,415.807119,1059.785770,1059.785770
640,640,hyogo__ashiya-kana__ashiya____2004-03-01,34.741667,135.306667,[ 34.74166667 135.30666667],11.225280,11.225280,11.225280,11.225280,417.310444,...,416.008686,416.008686,418.388330,418.388330,413.205938,413.205938,413.205938,413.205938,1060.345950,1060.345950
656,656,hyogo__kobe-shioya-kana__kouteoshaoya____2004-...,34.641667,135.075000,[ 34.64166667 135.075 ],12.726219,12.726219,12.726219,12.726219,440.753987,...,439.570057,439.570057,441.874804,441.874804,436.665314,436.665314,436.665314,436.665314,1080.106280,1080.106280
662,662,hyogo__nishinomiya-kana__nishinomiya,34.750000,135.335000,[ 34.75 135.335],13.959404,13.959404,13.959404,13.959404,414.563800,...,413.257603,413.257603,415.639914,415.639914,410.458635,410.458635,410.458635,410.458635,1058.322652,1058.322652
661,661,hyogo__naoki-kana__nagio____2006-03-06,34.830000,135.293333,[ 34.83 135.29333333],17.328916,17.328916,17.328916,17.328916,416.003084,...,414.444535,414.444535,416.990507,416.990507,411.867514,411.867514,411.867514,411.867514,1052.251858,1052.251858
658,658,hyogo__miki-kana__miki,34.776667,135.013333,[ 34.77666667 135.01333333],19.167264,19.167264,19.167264,19.167264,442.294360,...,440.714628,440.714628,443.276244,443.276244,438.157206,438.157206,438.157206,438.157206,1069.659912,1069.659912
641,641,hyogo__awaji-cho-kana__awajiko____2004-03-01,34.588333,135.016667,[ 34.58833333 135.01666667],20.403430,20.403430,20.403430,20.403430,447.560072,...,446.476910,446.476910,448.716447,448.716447,443.484825,443.484825,443.484825,443.484825,1087.852714,1087.852714
660,660,hyogo__mita-kana__sanda,34.895000,135.211667,[ 34.895 135.21166667],22.209597,22.209597,22.209597,22.209597,421.586981,...,419.798373,419.798373,422.493179,422.493179,417.425689,417.425689,417.425689,417.425689,1049.433814,1049.433814


In [116]:
air_nearest_weather = air_nearest_weather.drop(["latitude_str", "longitude_str", "station_id", "station_latitude",\
                                               "station_longitude", "station_vincenty", "station_great_circle"], 1)

In [118]:
air_nearest_weather.to_csv("air_nearest_weather_processed.csv")

## You can start reading the restaurant data from here

In [140]:
air_rest_with_stations = pd.DataFrame(pd.read_csv("air_nearest_weather_processed.csv"))

In [141]:
air_rest_with_stations.head()

,Unnamed: 0,air_store_id,air_genre_name,air_area_name,latitude,longitude,latlong_m,1_close_st,2_close_st,3_close_st,4_close_st,5_close_st
0,2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,[ 34.6951242 135.1978525],hyogo__kobe-kana__koube,hyogo__kobe-airport-kana__kobuteuko,hyogo__rokko-mountain-kana__rokkou-san____2006...,hyogo__ashiya-kana__ashiya____2004-03-01,hyogo__kobe-shioya-kana__kouteoshaoya____2004-...
1,3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,[ 34.6951242 135.1978525],hyogo__kobe-kana__koube,hyogo__kobe-airport-kana__kobuteuko,hyogo__rokko-mountain-kana__rokkou-san____2006...,hyogo__ashiya-kana__ashiya____2004-03-01,hyogo__kobe-shioya-kana__kouteoshaoya____2004-...
2,4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,[ 35.6580681 139.7515992],tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
3,5,air_99c3eae84130c1cb,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,[ 35.6580681 139.7515992],tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
4,6,air_f183a514cb8ff4fa,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,[ 35.6580681 139.7515992],tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi


In [165]:
air_rest_with_stations = air_rest_with_stations.set_index("air_store_id")

In [166]:
air_rest_with_stations.head()

,Unnamed: 0,air_genre_name,air_area_name,latitude,longitude,latlong_m,1_close_st,2_close_st,3_close_st,4_close_st,5_close_st
air_store_id,,,,,,,,,,,
air_fee8dcf4d619598e,2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,[ 34.6951242 135.1978525],hyogo__kobe-kana__koube,hyogo__kobe-airport-kana__kobuteuko,hyogo__rokko-mountain-kana__rokkou-san____2006...,hyogo__ashiya-kana__ashiya____2004-03-01,hyogo__kobe-shioya-kana__kouteoshaoya____2004-...
air_a17f0778617c76e2,3,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,[ 34.6951242 135.1978525],hyogo__kobe-kana__koube,hyogo__kobe-airport-kana__kobuteuko,hyogo__rokko-mountain-kana__rokkou-san____2006...,hyogo__ashiya-kana__ashiya____2004-03-01,hyogo__kobe-shioya-kana__kouteoshaoya____2004-...
air_83db5aff8f50478e,4,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,[ 35.6580681 139.7515992],tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
air_99c3eae84130c1cb,5,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,[ 35.6580681 139.7515992],tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
air_f183a514cb8ff4fa,6,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,[ 35.6580681 139.7515992],tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi


## Daily Visitor Data

indexed by rest_id and days. Cols include day_of_week, holiday_indicator, weather_measurements (multiple TBD), num_visitors

In [138]:
air_visit.loc["air_ba937bf13d40fb24"].index.values[:5]

array(['2016-01-13', '2016-01-14', '2016-01-15', '2016-01-16', '2016-01-18'], dtype=object)

In [142]:
air_visit.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [ ]:
## her bir gun icin en yakin 3 istasyondan average temp, wind speed (if available) ve precipitation datasi al
## her gun icin day of week datasi ve holiday indicator al 
## visitors zaten datanin icinde

In [146]:
date_info = date_info.set_index("calendar_date")

In [147]:
date_info.head()

,day_of_week,holiday_flg
calendar_date,,
2016-01-01,Friday,1
2016-01-02,Saturday,1
2016-01-03,Sunday,1
2016-01-04,Monday,0
2016-01-05,Tuesday,0


In [162]:
air_visit["day_of_week"] = air_visit["visit_date"].map(lambda x: date_info["day_of_week"][x])
air_visit["holiday_flg"] = air_visit["visit_date"].map(lambda x: date_info["holiday_flg"][x])

In [163]:
air_visit.head()

,air_store_id,visit_date,visitors,day_of_week,holiday_flg
0,air_ba937bf13d40fb24,2016-01-13,25,Wednesday,0
1,air_ba937bf13d40fb24,2016-01-14,32,Thursday,0
2,air_ba937bf13d40fb24,2016-01-15,29,Friday,0
3,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0
4,air_ba937bf13d40fb24,2016-01-18,6,Monday,0


In [177]:
air_rest_with_stations["1_close_st"]["air_fee8dcf4d619598e"]

'hyogo__kobe-kana__koube'

In [195]:
air_visit = air_visit[air_visit["air_store_id"]!="air_7cc17a324ae5c7dc"]

In [196]:
air_visit = air_visit[air_visit["air_store_id"]!="air_0f0cdeee6c9bf3d7"]

In [203]:
air_visit["1_closest_stat"] = air_visit["air_store_id"].map(lambda x: air_rest_with_stations["1_close_st"][x])
air_visit["2_closest_stat"] = air_visit["air_store_id"].map(lambda x: air_rest_with_stations["2_close_st"][x])
air_visit["3_closest_stat"] = air_visit["air_store_id"].map(lambda x: air_rest_with_stations["3_close_st"][x])
air_visit["4_closest_stat"] = air_visit["air_store_id"].map(lambda x: air_rest_with_stations["4_close_st"][x])
air_visit["5_closest_stat"] = air_visit["air_store_id"].map(lambda x: air_rest_with_stations["5_close_st"][x])

In [204]:
air_visit.head()

,air_store_id,visit_date,visitors,day_of_week,holiday_flg,closest_stat,1_closest_stat,2_closest_stat,3_closest_stat,4_closest_stat,5_closest_stat
2,air_ba937bf13d40fb24,2016-01-15,29,Friday,0,tokyo__tokyo-kana__tonokyo,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
3,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0,tokyo__tokyo-kana__tonokyo,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
4,air_ba937bf13d40fb24,2016-01-18,6,Monday,0,tokyo__tokyo-kana__tonokyo,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
5,air_ba937bf13d40fb24,2016-01-19,9,Tuesday,0,tokyo__tokyo-kana__tonokyo,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
6,air_ba937bf13d40fb24,2016-01-20,31,Wednesday,0,tokyo__tokyo-kana__tonokyo,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi


In [205]:
air_visit = air_visit.drop("closest_stat", 1)
air_visit.to_csv("air_visit_wdates.csv")

In [243]:
air_visit_try = pd.DataFrame(pd.read_csv("air_visit_wdates.csv"))

In [244]:
air_visit_try.head()

,Unnamed: 0,air_store_id,visit_date,visitors,day_of_week,holiday_flg,1_closest_stat,2_closest_stat,3_closest_stat,4_closest_stat,5_closest_stat
0,2,air_ba937bf13d40fb24,2016-01-15,29,Friday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
1,3,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
2,4,air_ba937bf13d40fb24,2016-01-18,6,Monday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
3,5,air_ba937bf13d40fb24,2016-01-19,9,Tuesday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
4,6,air_ba937bf13d40fb24,2016-01-20,31,Wednesday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi


In [246]:
# pd.DataFrame(pd.read_csv('data/rrv-weather-data/measurements/tokyo__tokyo-kana__tonokyo.csv'))

## Merging for Weather Measurements

Will take into account the data of all restaurants as separate DataFrames, then merge each of them with the measurements of corresponding stations, then concat all of them along with the y-axis

In [307]:
air_ba937bf13d40fb24_df = (pd.DataFrame(air_visit_try[air_visit_try["air_store_id"]=="air_ba937bf13d40fb24"]).\
join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(air_visit_try[air_visit_try["air_store_id"]=="air_ba937bf13d40fb24"]["1_closest_stat"])[-1] + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="0", rsuffix="1")).drop(["Unnamed: 00", "Unnamed: 01"],1)

In [308]:
# air_ba937bf13d40fb24_df

In [309]:
air_ba937bf13d40fb24_df = air_ba937bf13d40fb24_df.join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(air_visit_try[air_visit_try["air_store_id"]=="air_ba937bf13d40fb24"]["1_closest_stat"])[-1] + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="1", rsuffix="2").drop(["Unnamed: 0"],1)

In [310]:
# air_ba937bf13d40fb24_df

In [311]:
air_ba937bf13d40fb24_df = air_ba937bf13d40fb24_df.join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(air_visit_try[air_visit_try["air_store_id"]=="air_ba937bf13d40fb24"]["1_closest_stat"])[-1] + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="2", rsuffix="3").drop(["Unnamed: 0"],1)


In [312]:
air_ba937bf13d40fb24_df

,air_store_id,visit_date,visitors,day_of_week,holiday_flg,1_closest_stat,2_closest_stat,3_closest_stat,4_closest_stat,5_closest_stat,...,cloud_cover2,avg_temperature,high_temperature,low_temperature,hours_sunlight,solar_radiation,avg_wind_speed,avg_local_pressure,avg_sea_pressure,cloud_cover
0,air_ba937bf13d40fb24,2016-01-15,29,Friday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,6.0,5.6,10.9,2.0,8.1,11.67,2.5,1013.1,1016.1,6.0
1,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,2.3,6.5,11.8,1.8,9.1,12.41,1.9,1015.8,1018.8,2.3
2,air_ba937bf13d40fb24,2016-01-18,6,Monday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,7.5,2.8,6.2,0.2,1.4,2.40,3.7,995.9,998.9,7.5
3,air_ba937bf13d40fb24,2016-01-19,9,Tuesday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,1.5,5.1,8.6,0.9,9.1,13.37,4.0,997.8,1000.7,1.5
4,air_ba937bf13d40fb24,2016-01-20,31,Wednesday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,1.8,4.1,9.1,-0.7,9.2,13.21,4.3,1006.3,1009.2,1.8
5,air_ba937bf13d40fb24,2016-01-21,21,Thursday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,5.3,5.4,9.5,1.0,7.4,10.35,3.1,1014.4,1017.5,5.3
6,air_ba937bf13d40fb24,2016-01-22,18,Friday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,5.5,5.8,10.3,3.0,8.1,12.72,3.1,1016.5,1019.5,5.5
7,air_ba937bf13d40fb24,2016-01-23,26,Saturday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,10.0,3.9,6.4,2.6,0.2,4.77,2.5,1014.4,1017.4,10.0
8,air_ba937bf13d40fb24,2016-01-25,21,Monday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,0.0,2.9,8.4,-2.6,9.6,14.01,2.9,1013.5,1016.6,0.0
9,air_ba937bf13d40fb24,2016-01-26,11,Tuesday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,0.0,3.8,9.6,-1.3,8.6,12.10,2.0,1019.7,1022.7,0.0


In [313]:
air_ba937bf13d40fb24_df = air_ba937bf13d40fb24_df.join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(air_visit_try[air_visit_try["air_store_id"]=="air_ba937bf13d40fb24"]["1_closest_stat"])[-1] + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="3", rsuffix="4").drop(["Unnamed: 0"],1)

In [314]:
air_ba937bf13d40fb24_df

,air_store_id,visit_date,visitors,day_of_week,holiday_flg,1_closest_stat,2_closest_stat,3_closest_stat,4_closest_stat,5_closest_stat,...,cloud_cover3,avg_temperature4,high_temperature4,low_temperature4,hours_sunlight4,solar_radiation4,avg_wind_speed4,avg_local_pressure4,avg_sea_pressure4,cloud_cover4
0,air_ba937bf13d40fb24,2016-01-15,29,Friday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,6.0,5.6,10.9,2.0,8.1,11.67,2.5,1013.1,1016.1,6.0
1,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,2.3,6.5,11.8,1.8,9.1,12.41,1.9,1015.8,1018.8,2.3
2,air_ba937bf13d40fb24,2016-01-18,6,Monday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,7.5,2.8,6.2,0.2,1.4,2.40,3.7,995.9,998.9,7.5
3,air_ba937bf13d40fb24,2016-01-19,9,Tuesday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,1.5,5.1,8.6,0.9,9.1,13.37,4.0,997.8,1000.7,1.5
4,air_ba937bf13d40fb24,2016-01-20,31,Wednesday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,1.8,4.1,9.1,-0.7,9.2,13.21,4.3,1006.3,1009.2,1.8
5,air_ba937bf13d40fb24,2016-01-21,21,Thursday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,5.3,5.4,9.5,1.0,7.4,10.35,3.1,1014.4,1017.5,5.3
6,air_ba937bf13d40fb24,2016-01-22,18,Friday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,5.5,5.8,10.3,3.0,8.1,12.72,3.1,1016.5,1019.5,5.5
7,air_ba937bf13d40fb24,2016-01-23,26,Saturday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,10.0,3.9,6.4,2.6,0.2,4.77,2.5,1014.4,1017.4,10.0
8,air_ba937bf13d40fb24,2016-01-25,21,Monday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,0.0,2.9,8.4,-2.6,9.6,14.01,2.9,1013.5,1016.6,0.0
9,air_ba937bf13d40fb24,2016-01-26,11,Tuesday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi,...,0.0,3.8,9.6,-1.3,8.6,12.10,2.0,1019.7,1022.7,0.0


In [300]:
air_ba937bf13d40fb24_df = air_ba937bf13d40fb24_df.join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(air_visit_try[air_visit_try["air_store_id"]=="air_ba937bf13d40fb24"]["1_closest_stat"])[-1] + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="4", rsuffix="5").drop(["Unnamed: 0"],1)

In [317]:
def merge_rest_with_weather_data(data, rest_id):
    merged_data = (((((pd.DataFrame(air_visit_try[air_visit_try["air_store_id"]==rest_id]).\
join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(pd.DataFrame(data[data["air_store_id"]==rest_id])["1_closest_stat"])[-1] + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="0", rsuffix="1")).drop(["Unnamed: 00", "Unnamed: 01"],1)).join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(pd.DataFrame(data[data["air_store_id"]==rest_id])["1_closest_stat"])[-1]  + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="1", rsuffix="2").drop(["Unnamed: 0"],1)).join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(pd.DataFrame(data[data["air_store_id"]==rest_id])["1_closest_stat"])[-1] + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="2", rsuffix="3").drop(["Unnamed: 0"],1)).join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(pd.DataFrame(data[data["air_store_id"]==rest_id])["1_closest_stat"])[-1] + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="3", rsuffix="4").drop(["Unnamed: 0"],1)).join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(pd.DataFrame(data[data["air_store_id"]==rest_id])["1_closest_stat"])[-1] + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="4", rsuffix="5").drop(["Unnamed: 0"],1)
    return merged_data

In [319]:
# merge_rest_with_weather_data(air_visit_try, "air_ba937bf13d40fb24")

In [320]:
## merging the separate dataframes that belong to each restaurant with corresponding 5 weather
## center measurement data, and writing all in a dictionary where "rest_id" = dataframe object
df_weather = {}
for rest in list(set(air_visit_try["air_store_id"])):
    df_weather[rest] = merge_rest_with_weather_data(air_visit_try, rest)

In [329]:
df_weather["air_c8a657c8c5c93d69"].head()

,air_store_id,visit_date,visitors,day_of_week,holiday_flg,1_closest_stat,2_closest_stat,3_closest_stat,4_closest_stat,5_closest_stat,...,high_temperature,low_temperature,hours_sunlight,solar_radiation,avg_wind_speed,avg_vapor_pressure,avg_local_pressure,avg_humidity,avg_sea_pressure,cloud_cover
103696,air_c8a657c8c5c93d69,2016-01-05,3,Tuesday,0,fukuoka__fukuoka-kana__fukuoka,fukuoka__hakata-kana__hakata,fukuoka__sayara-wakayama-kana__sawarawakiyama,fukuoka__dazaifu-kana__dasaif,fukuoka__sasaguri-kana__sasaburi____2009-11-20,...,12.4,8.3,0.0,1.46,1.5,9.2,1021.2,76,1023.0,10.0
103697,air_c8a657c8c5c93d69,2016-01-06,2,Wednesday,0,fukuoka__fukuoka-kana__fukuoka,fukuoka__hakata-kana__hakata,fukuoka__sayara-wakayama-kana__sawarawakiyama,fukuoka__dazaifu-kana__dasaif,fukuoka__sasaguri-kana__sasaburi____2009-11-20,...,12.0,7.2,0.0,4.40,1.4,8.9,1019.4,76,1021.2,9.5
103698,air_c8a657c8c5c93d69,2016-01-08,10,Friday,0,fukuoka__fukuoka-kana__fukuoka,fukuoka__hakata-kana__hakata,fukuoka__sayara-wakayama-kana__sawarawakiyama,fukuoka__dazaifu-kana__dasaif,fukuoka__sasaguri-kana__sasaburi____2009-11-20,...,8.8,6.7,0.0,1.44,3.8,6.1,1019.2,59,1021.1,10.0
103699,air_c8a657c8c5c93d69,2016-01-09,2,Saturday,0,fukuoka__fukuoka-kana__fukuoka,fukuoka__hakata-kana__hakata,fukuoka__sayara-wakayama-kana__sawarawakiyama,fukuoka__dazaifu-kana__dasaif,fukuoka__sasaguri-kana__sasaburi____2009-11-20,...,9.1,7.1,0.2,3.80,2.4,5.8,1022.1,54,1024.0,10.0
103700,air_c8a657c8c5c93d69,2016-01-12,13,Tuesday,0,fukuoka__fukuoka-kana__fukuoka,fukuoka__hakata-kana__hakata,fukuoka__sayara-wakayama-kana__sawarawakiyama,fukuoka__dazaifu-kana__dasaif,fukuoka__sasaguri-kana__sasaburi____2009-11-20,...,9.2,5.0,0.4,3.27,2.2,6.9,1017.2,67,1019.1,8.0


In [330]:
df_weather["air_c8a657c8c5c93d69"].columns

Index(['air_store_id', 'visit_date', 'visitors', 'day_of_week', 'holiday_flg',
       '1_closest_stat', '2_closest_stat', '3_closest_stat', '4_closest_stat',
       '5_closest_stat', 'avg_temperature1', 'high_temperature1',
       'low_temperature1', 'hours_sunlight1', 'solar_radiation1',
       'avg_wind_speed1', 'avg_vapor_pressure1', 'avg_local_pressure1',
       'avg_humidity1', 'avg_sea_pressure1', 'cloud_cover1',
       'avg_temperature2', 'high_temperature2', 'low_temperature2',
       'hours_sunlight2', 'solar_radiation2', 'avg_wind_speed2',
       'avg_vapor_pressure2', 'avg_local_pressure2', 'avg_humidity2',
       'avg_sea_pressure2', 'cloud_cover2', 'avg_temperature3',
       'high_temperature3', 'low_temperature3', 'hours_sunlight3',
       'solar_radiation3', 'avg_wind_speed3', 'avg_vapor_pressure3',
       'avg_local_pressure3', 'avg_humidity3', 'avg_sea_pressure3',
       'cloud_cover3', 'avg_temperature4', 'high_temperature4',
       'low_temperature4', 'hours_sunlight

In [390]:
def select_weather_measurement(data):
    """Given a rest_id dataframe merged with weather measurements, 
    selects the weather measurement columns starting from the closest station.
    Final measurement features should include at least one column from each each 
    variable below: 
    - low_temperature,
    - high_temperature,
    - avg_temperature,
    - hours_sunlight."""
    ## The measurement types that are wanted in the data (from at least one station)
    required_measurements = ["hours_sunlight", "low_temperature", "high_temperature",\
                            "avg_temperature", "precipitation"]
    ## The core columns that should not be dropped before returning the data
    ## These are not weather measurements.
    core_columns = ["air_store_id", "visit_date", "visitors", "day_of_week",\
                   "holiday_flg", "1_closest_stat", "2_closest_stat", "3_closest_stat",\
                   "4_closest_stat", "5_closest_stat"]
    
    ## Initialize the list of column names that will be kept in the data
    for col in required_measurements:
        ## Check if the closest station has that measurement
        if col + "1" in data.columns.values:
            core_columns.append(str(col + "1"))
        else:
            if col + "2" in data.columns.values:
                core_columns.append(str(col + "2"))
            else:
                if col + "3" in data.columns.values:
                    core_columns.append(str(col + "3"))
                else:
                    if col + "4" in data.columns.values:
                        core_columns.append(str(col + "4"))
                    else:
                        if col in data.columns.values:
                            core_columns.append(col)

    return data[core_columns]

In [394]:
selected = {}
selected_all = {}
for rest in list(df_weather.keys()):
    selected[rest] = select_weather_measurement(df_weather[rest])
    if ("hours_sunlight" or "hours_sunlight1" or "hours_sunlight2"or "hours_sunlight3" or "hours_sunlight4" in selected[rest].columns.values) and ("low_temperature" or "low_temperature1" or "low_temperature2" or "low_temperature3"or "low_temperature4" in selected[rest].columns.values)\
    and ("high_temperature" or "high_temperature1" or "high_temperature2" or "high_temperature3" or "high_temperature4" in selected[rest].columns.values) and ("avg_temperature" or "avg_temperature1" or "avg_temperature2" or "avg_temperature3" or "avg_temperature4" in selected[rest].columns.values)\
    and ("precipitation" or "precipitation1" or "precipitation2" or "precipitation3" or "precipitation4" in selected[rest].columns.values):
        selected_all[rest] = selected[rest]

In [395]:
len(selected_all.keys())

827

In [397]:
selected_all["air_8110d68cc869b85e"]

,air_store_id,visit_date,visitors,day_of_week,holiday_flg,1_closest_stat,2_closest_stat,3_closest_stat,4_closest_stat,5_closest_stat,hours_sunlight1,low_temperature1,high_temperature1,avg_temperature1
151878,air_8110d68cc869b85e,2017-01-24,4,Tuesday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,tokyo__nerima-kana__nerima,9.4,-1.9,7.7,2.5
151879,air_8110d68cc869b85e,2017-02-01,28,Wednesday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,tokyo__nerima-kana__nerima,7.2,2.2,10.6,6.0
151880,air_8110d68cc869b85e,2017-02-02,1,Thursday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,tokyo__nerima-kana__nerima,9.6,0.9,9.7,4.5
151881,air_8110d68cc869b85e,2017-02-03,14,Friday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,tokyo__nerima-kana__nerima,9.6,1.0,13.7,7.4
151882,air_8110d68cc869b85e,2017-02-04,52,Saturday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,tokyo__nerima-kana__nerima,9.6,4.4,13.4,8.2
151883,air_8110d68cc869b85e,2017-02-05,19,Sunday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,tokyo__nerima-kana__nerima,2.5,3.4,9.9,7.2
151884,air_8110d68cc869b85e,2017-02-06,15,Monday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,tokyo__nerima-kana__nerima,6.1,3.3,15.7,8.7
151885,air_8110d68cc869b85e,2017-02-08,13,Wednesday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,tokyo__nerima-kana__nerima,9.5,1.9,11.0,6.3
151886,air_8110d68cc869b85e,2017-02-10,9,Friday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,tokyo__nerima-kana__nerima,4.0,1.5,9.3,4.3
151887,air_8110d68cc869b85e,2017-02-11,38,Saturday,1,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,tokyo__nerima-kana__nerima,7.3,-1.6,10.1,4.1


In [400]:
merged_data_final

,1_closest_stat,2_closest_stat,3_closest_stat,4_closest_stat,5_closest_stat,air_store_id,avg_temperature1,day_of_week,high_temperature1,holiday_flg,hours_sunlight1,low_temperature1,precipitation1,visit_date,visitors
228949,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,9.3,Sunday,16.2,1,8.3,3.0,NaN,2016-01-03,10
228950,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,9.2,Monday,15.2,0,8.9,4.4,NaN,2016-01-04,9
228951,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,10.9,Tuesday,15.4,0,8.4,5.1,NaN,2016-01-05,17
228952,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,8.9,Wednesday,11.3,0,0.0,7.1,NaN,2016-01-06,10
228953,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,6.8,Friday,10.9,0,8.7,3.1,NaN,2016-01-08,5
228954,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,7.3,Saturday,11.6,0,8.9,2.8,NaN,2016-01-09,8
228955,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,7.9,Sunday,12.9,0,9.1,2.4,NaN,2016-01-10,16
228956,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,7.1,Monday,10.5,1,3.3,3.8,NaN,2016-01-11,16
228957,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,3.4,Tuesday,6.1,0,0.0,0.9,NaN,2016-01-12,8
228958,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,4.3,Wednesday,9.7,0,8.7,-0.9,NaN,2016-01-13,9


In [399]:
merged_data_final = pd.concat([select_weather_measurement(df_weather[rest]) for rest in df_weather.keys()])
# select_weather_measurement(df_weather["air_ba937bf13d40fb24"])
merged_data_final.to_csv("merged_rest_data_weather_measurements.csv")

In [389]:
merged_data_final.columns

Index(['1_closest_stat', '2_closest_stat', '3_closest_stat', '4_closest_stat',
       '5_closest_stat', 'air_store_id', 'avg_temperature1', 'day_of_week',
       'high_temperature1', 'holiday_flg', 'hours_sunlight1',
       'low_temperature1', 'visit_date', 'visitors'],
      dtype='object')

In [ ]:
meteo_weights = {"2": }

In [210]:
import os
def dropcols(data):
    data = data.dropna(axis=1, how='all')

In [211]:
## Writing all the measurements to a filelist (there are many measurements)
def air_station_list(path):
    file_list = os.listdir(path)[1:]
    return file_list

In [212]:
## overwriting the csv files without the nan columns
for measurement_stat in air_station_list('data/rrv-weather-data/measurements/'): 
    ## get the full path
    station_path = 'data/rrv-weather-data/measurements/' + measurement_stat
    ## read each file to a df, drop nan cols and overwrite
    pd.DataFrame(pd.read_csv(station_path)).dropna(axis=1, how='all').to_csv(station_path, index=False)

In [234]:
for i in range(1,6):
    air_visit_try[str(i) + "_precip"] = [np.nan]*len(air_visit_try)
    air_visit_try[str(i) + "_avg_temp"] = [np.nan]*len(air_visit_try)
    air_visit_try[str(i) + "_low_temp"] = [np.nan]*len(air_visit_try)
    air_visit_try[str(i) + "_hi_temp"] = [np.nan]*len(air_visit_try)
    air_visit_try[str(i) + "_avg_windsp"] = [np.nan]*len(air_visit_try)

In [ ]:
air_visit_try.columns

In [239]:
# for i in range(1,6):
#     air_visit_try[str(i) + "_precip"] = air_visit_try[str(i) + "_closest_stat"].\
#     map(lambda x: pd.DataFrame(pd.read_csv('data/rrv-weather-data/measurements/' + str(x) + ".csv"))["precipitation"])

In [242]:
air_visit_try.head()
## if cols in colindex
## air_visit_try.to_csv("air_visit_wmindicators.csv")
## if cols not in colindex
## get_measurement_indicators(air_visit_try).to_csv("air_visit_wmindicators.csv")

,Unnamed: 0,air_store_id,visit_date,visitors,day_of_week,holiday_flg,1_closest_stat,2_closest_stat,3_closest_stat,4_closest_stat,...,5_avg_temp,5_low_temp,5_hi_temp,5_avg_windsp,1_closest_stat_precip,1_closest_stat_avg_temp,1_closest_stat_high_temp,1_closest_stat_low_temp,1_closest_stat_wind_spd,2_closest_stat_precip
0,2,air_ba937bf13d40fb24,2016-01-15,29,Friday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,...,NaN,NaN,NaN,NaN,0,1,1,1,1,1
1,3,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,...,NaN,NaN,NaN,NaN,0,1,1,1,1,1
2,4,air_ba937bf13d40fb24,2016-01-18,6,Monday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,...,NaN,NaN,NaN,NaN,0,1,1,1,1,1
3,5,air_ba937bf13d40fb24,2016-01-19,9,Tuesday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,...,NaN,NaN,NaN,NaN,0,1,1,1,1,1
4,6,air_ba937bf13d40fb24,2016-01-20,31,Wednesday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,...,NaN,NaN,NaN,NaN,0,1,1,1,1,1


In [ ]:
## bunun yerine 5ini ayirt eden fonksiyon yaz
def get_closest_stat(data):
    """For air_visit_try that includes the names of 
    5 closest stations to each restaurant.
    - avg_temperature
    - avg_wind_speed
    - low_temperature
    - high_temperature
    - precipitation"""
    for i in range(1,6):
        air_visit_try[str(i) + "_precip"] = air_visit_try[str(i) + "_closest_stat"].\
        map(lambda x: pd.DataFrame(pd.read_csv('data/rrv-weather-data/measurements/' + str(x)))["precipitation"])
    
        
    

In [222]:
for store in air_visit_try["air_store_id"]:
    closest_station_path = 'data/rrv-weather-data/measurements/' + list(air_visit_try[air_visit_try["air_store_id"]==store]["1_closest_stat"])[0] + ".csv"
    weather_data = pd.DataFrame(pd.read_csv(closest_station_path))
    air_visit_try[air_visit_try["air_store_id"]==store]["temp1"] = air_visit_try["1_closest_stat"].map(lambda x: weather_data["avg_temperature"])

station_path = 'data/rrv-weather-data/measurements/' + measurement_stat

/Users/derin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



KeyError: 'avg_temperature'

In [214]:
air_visit_try[air_visit_try["air_store_id"]=="air_ba937bf13d40fb24"]["1_closest_stat"]

0      tokyo__tokyo-kana__tonokyo
1      tokyo__tokyo-kana__tonokyo
2      tokyo__tokyo-kana__tonokyo
3      tokyo__tokyo-kana__tonokyo
4      tokyo__tokyo-kana__tonokyo
5      tokyo__tokyo-kana__tonokyo
6      tokyo__tokyo-kana__tonokyo
7      tokyo__tokyo-kana__tonokyo
8      tokyo__tokyo-kana__tonokyo
9      tokyo__tokyo-kana__tonokyo
10     tokyo__tokyo-kana__tonokyo
11     tokyo__tokyo-kana__tonokyo
12     tokyo__tokyo-kana__tonokyo
13     tokyo__tokyo-kana__tonokyo
14     tokyo__tokyo-kana__tonokyo
15     tokyo__tokyo-kana__tonokyo
16     tokyo__tokyo-kana__tonokyo
17     tokyo__tokyo-kana__tonokyo
18     tokyo__tokyo-kana__tonokyo
19     tokyo__tokyo-kana__tonokyo
20     tokyo__tokyo-kana__tonokyo
21     tokyo__tokyo-kana__tonokyo
22     tokyo__tokyo-kana__tonokyo
23     tokyo__tokyo-kana__tonokyo
24     tokyo__tokyo-kana__tonokyo
25     tokyo__tokyo-kana__tonokyo
26     tokyo__tokyo-kana__tonokyo
27     tokyo__tokyo-kana__tonokyo
28     tokyo__tokyo-kana__tonokyo
29     tokyo__